In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()


In [20]:
import pandas as pd
import requests
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ").replace("\u00A0", " ")  # Non-breaking Space 제거
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + OPENAI_API_KEY
    }
    json_data = {
        'input': text,
        'model': model,
    }
    try:
        response = requests.post('https://api.openai.com/v1/embeddings', headers=headers, json=json_data)
        if response.status_code == 200:
            embedding = response.json()['data'][0]['embedding']
            return embedding
        else:
            print(f"임베딩 가져오기 오류: HTTP {response.status_code} - {response.text}")
            return None
    except Exception as e:
        # 다른 예외 처리
        print(f"임베딩 가져오기 오류: {e}")
        return None

# 데이터프레임 로드
df = pd.read_excel('./data/naver_news.xlsx')

# 첫 번째 행의 제목 추출
sample_text = df.loc[0, '제목']
print("Sample Text:", sample_text)

# 임베딩 추출
embedding = get_embedding(sample_text)
print("Embedding:", embedding)


Sample Text: 뉴타운급 재개발·재건축, 수요자 관심 높아져
Embedding: [-0.022099858, 0.006423026, 0.058312938, 0.04154913, -0.021286082, 0.0033626428, -0.00038908713, 0.037154734, -0.024111051, -0.041781638, 0.023262398, 0.005617968, 0.018333234, 0.013206438, 0.04905913, -0.029109966, -0.026645385, -0.08193572, -0.026598884, -0.018705247, 0.026529131, -0.01819373, 0.014334101, -0.025389843, 0.023646036, 0.042269904, 0.0068880413, -0.007457685, -0.02363441, -0.0031156035, -0.0034876158, -0.036619965, -0.008108707, 0.021413961, 0.0099629555, 0.0346669, 0.028342692, -0.021809224, -0.010416346, 0.020111918, -0.0028467665, -0.008846919, -0.006266083, -0.009195681, 0.005862101, 0.019437645, -0.041665383, 0.03494591, -0.013590076, 0.048036095, 0.021948729, -0.01295068, 0.03527142, -0.020995447, 0.02439006, 0.03643396, 0.007556501, -0.0330626, 0.02683139, -0.020042166, -0.00044721406, -0.040781856, 0.010613978, 0.011084805, -0.0123926615, -0.02736616, 0.023367027, 0.016496424, -0.034806404, 0.00738212, 0.013659828

In [24]:
import openai
api_key = OPENAI_API_KEY

# OpenAI API 클라이언트 초기화
openai.api_key = api_key

response = openai.chat.completions.create(
     model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "당신은 창작 작가 입니다."},
        {"role": "user", "content": "말하는 고양이에 대한 짧은 이야기를 작성하시오"}
    ],
    temperature=0.8,
    max_tokens=500,
    top_p=1.0,
    frequency_penalty=1.0,
    presence_penalty=1.0
)

print("Assistant Response:")
print(response.choices[0].message)


Assistant Response:
ChatCompletionMessage(content="옛날 한 마을에 특별한 고양이가 살고 있었습니다. 이름은 미야. 미야는 일반적인 고양이와 달리 사람의 말을 할 수 있는 능력을 가지고 있었죠. 그래서 마을 사람들은 그녀를 '말하는 고양이'라고 불렀습니다.\n\n미야는 매일 아침 동네 공원에서 산책하며 사람들과 대화를 나누곤 했습니다. 아이들은 신나게 미야에게 다가와 질문했습니다.\n\n“미야, 왜 이렇게 귀여워요?” \n미야는 웃으며 대답했어요. “그건 내가 사랑스럽기 때문이지!”\n\n주인공 소녀, 지아도 매일 미야를 찾았습니다. 지아는 학교에서 힘든 일들이 많았고, 항상 혼자였거든요. 어느 날, 지아가 울면서 말했어요.\n“나는 친구가 없어… 모두 나를 싫어해.”\n\n미야는 부드러운 목소리로 말했다.\n“지아, 세상에는 너를 좋아할 사람들이 많단다. 네 안에 숨겨진 빛을 보여줘.”\n\n지아는 그 말이 위안이 되었고, 조금씩 자신감을 되찾기 시작했습니다. 이후 두 친구 사이에는 진정한 우정이 피어났습니다.\n\n시간이 지나자 지아도 다른 친구들을 사귀게 되었고, 마을 사람들도 서로 더 따뜻하게 대하기 시작했습니다. 이제 모든 이들은 미ya 덕분에 마음의 문을 열게 된 것이었죠.\n\n그리고 그렇게 말하는 고양이는 언제까지나 마을의 사랑받는 존재로 남았습니다.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [25]:
with open('./data/nomuhun.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()


response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Generate an executive summary focusing on key points and actionable insights."},
        {"role": "user", "content": f"문서를 3문장으로 요약해주세요: {file_content}"}
    ],
    temperature=0.2,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0
) 

print("Original Text:\n", file_content)
print("Summary Text:\n", response.choices[0].message.content)


Original Text:
 시간이 많이 됐는데, 상생에 관해서 한 말씀 드리겠습니다. 좋은 겁니다. 대화와 타협의 문화가 바로 상생입니다. 상생은 그야말로 진실하게 이것을 실천할 의지가 있어야 됩니다. 상대방에게 양보를 받아내기 위해서 상대방을 공격하기 위해서 상생을 내세우면 그 상생은 반드시 실패합니다.
어떻게 하는 것이 상생인지를 알아야 합니다. 세상이 변화할 때는 변화를 수용할 줄 알아야 하고, 기득권을 버려야 할 때는 기득권을 버려야 합니다. 새로운 문화를 장려해야 될 때 낡은 문화를 고집하면 안 됩니다. 시대의 흐름도 맞추어야합니다.그 다음에 상생을 하는 기본조건을 갖추어야 됩니다. 상대를 존중할 줄 알아야합니다. 배제의 습관이 남아서, 지금도 계속 배제하려고 하는 방법으로는 상생할수 없습니다. 상생은 결국 대화, 토론, 설득, 타협, 그리고 거의 다 합의가 된 것같은데 마지막 결론이 안 날 때 그때 표결하는 겁니다. 그렇죠? 표결하고 승복하는 겁니다. 승복해야 상생이 이루어질 수 있는 것 아닙니까? 그 규칙을 무시하면상생이 안 됩니다.  스포츠 게임도 규칙을 잘 지키고 끝났을 때 두 사람이 서로 악수하고 그렇게하지 않습니까? 반칙으로 얼룩진 경기가 끝났을 때 무슨 상생이 되겠습니까? 규칙과 승복, 훌륭한 심판 매우 중요합니다. 패배를 넉넉하게 수용할 줄 아는 그런역량을 갖추고 싶습니다. 저는 지금도 열심히 일하고 있습니다만, 어떻든 권력을 추구한 사람으로서는이제 하산길로 들어서고 있습니다. 발 삐지 않고 무사히 하산을 잘했으면 좋겠습니다. 등산은 올라갈 때보다 내려갈 때가 더 위험하다고 합니다. 무사하게 하산하기위해서는 정상의 경치에 대해서 미련을 갖지 않아야 합니다. 정상의 경치가 저에게는 좋기도 하지만 골치 아픈 일도 많습니다. 미련을 갖지 않겠습니다. 이것은말로 되는 것이 아니고 끊임없이 자기와의 승부 속에서 가능한 일입니다. 제 자신이 여유 있는 마음으로 하산할 수 있도록 자신을 다스려 내는 것, 그것이 제가 해야 될 남은 일입니다.감사합니

In [26]:
def get_movie_recommendation(age, gender, genre, country, mood) :
    prompt = f"""
    당신은 영화 추천 전문가입니다. 아래 사용자의 정보를 기반으로 개인 맞춤형 영화 5편을 추천해주세요
    각 영화의 제목, 개봉 연도, 감독, 주연배우, 간단한 줄거리를 포함해주세요
    - 나이 : {age}
    - 성별 : {gender}
    - 선호 장르 : {genre}
    - 한국영화 또는 외국영화 : {country}
    - 감정 상태 : {mood}
    
    개인 맞춤형 영화 추천해주세요"""

    try :
        response = openai.chat.completions.create(
            model ="gpt-4o-mini",
            messages =[ { "role": "system", "content" : "당신은 영화 추천 전문가입니다"},
                        { "role" : "user", "content" : prompt}  ]
        )
        recommendations = response.choices[0].message.content
        return recommendations
    except  openai.error.OpenAIError as e :
        return f"오류가 발생했습니다 : {str(e)}"       

In [27]:
age = input("나이를 입력하세요")
gender = input("성별 입력하세요 (예: 남성, 여성)")
genre = input("선호하는 장르를 입력하세요(예 : 액션, 코미디, 로맨스)")
country = input("어느 나라 영화를 선호하시나요? (예: 한국, 미국, 중국, 일본)")
mood = input("현재 감정상태를 입력하세요(예: 행복, 우울, ")
recommendations = get_movie_recommendation(age, gender, genre, country, mood)
print("**** 개인 맞춤 영화 추천 리스트 ****")
print(recommendations)

나이를 입력하세요 28
성별 입력하세요 (예: 남성, 여성) 남성
선호하는 장르를 입력하세요(예 : 액션, 코미디, 로맨스) 감정이 매마른사람도 눈물이나는 감동적인
어느 나라 영화를 선호하시나요? (예: 한국, 미국, 중국, 일본) 한국
현재 감정상태를 입력하세요(예: 행복, 우울,  우울


**** 개인 맞춤 영화 추천 리스트 ****
감정이 매마른 사람도 눈물을 흘리게 만드는 감동적인 한국 영화를 추천해드리겠습니다. 아래의 5편을 확인해보세요.

1. **제목:** 아이 캔 스피크 (I Can Speak)  
   **개봉 연도:** 2017  
   **감독:** 김현석  
   **주연 배우:** 나문희, 이제훈  
   **줄거리:** 1960년대 일본군 위안부 문제와 한 노인이 공무원과의 특별한 관계를 통해 자신의 아픈 과거를 극복해 나가는 이야기를 그립니다. 세대 간의 소통과 감정을 풀어가는 과정에서 진한 감동이 느껴집니다.

2. **제목:** 그때 그 이야기 (The Way Back)  
   **개봉 연도:** 2022  
   **감독:** 장재현  
   **주연 배우:** 이성민, 김희애  
   **줄거리:** 한 남성이 과거의 상처를 극복하고 새로운 삶을 찾아가는 과정을 그린 이야기로, 가족과의 관계에서 오가는 복잡한 감정들이 깊이 있게 묘사됩니다. 어른들의 복잡한 감정을 진솔하게 담아낸 작품입니다.

3. **제목:** 우리 생애 최고의 순간 (Our Measure of a Man)  
   **개봉 연도:** 2008  
   **감독:** 이정향  
   **주연 배우:** 이영애, 김명민  
   **줄거리:** 전직 체조선수인 주인공이 인생의 위기를 극복하고 재기하기 위해 힘쓰는 여정을 감동적으로 그렸습니다. 삶의 의미와 진정한 성공에 대한 메시지를 전달하며, 감정적으로 깊은 여운을 남깁니다.

4. **제목:** 너의 췌장을 먹고 싶어 (I Want to Eat Your Pancreas)  
   **개봉 연도:** 2017  
   **감독:** 엄유나  
   **주연 배우:** 안재홍, 다혜  
   **줄거리:** 불치병에 걸린 소녀와 그녀의 비밀을 알게 된 남자애의 우정과 사랑을 그린 영화로, 삶과 죽음의 경계에서 진정한 우정의 의미를 되새기게 하는 작품입니다. 눈물 없는 관람은 힘든 이야기입니다.

